In [1]:
from astropy.table import Table, vstack
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as au
import glob

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir

print(sample_table_file)

sample_table = Table.read(sample_table_file)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits


In [3]:
sample_table_ = []
gals = ['NGC3627', 'NGC4321', 'NGC6946']

for i, gal in enumerate(gals):
    mask = sample_table['name'] == gal.lower()
    sample_table_ += sample_table[mask]
    
sample_table = vstack(sample_table_)

In [4]:
mass_co = (sample_table['lco_phangs'].quantity * sample_table['appcor_phangs'].quantity * sample_table['aco10_phangs'].quantity) / 0.65
sample_table['props_mmol'] = mass_co

In [5]:
sample_table

name,pgc,alias,survey_astrosat_status,survey_astrosat_instrument,survey_astrosat_notes,survey_astrosat_references,survey_galex_status,survey_galex_instrument,survey_galex_notes,survey_galex_references,survey_halpha_status,survey_halpha_instrument,survey_halpha_notes,survey_halpha_references,survey_muse_status,survey_muse_instrument,survey_muse_notes,survey_muse_references,survey_kcwi_status,survey_kcwi_instrument,survey_kcwi_notes,survey_kcwi_references,survey_sitelle_status,survey_sitelle_instrument,survey_sitelle_notes,survey_sitelle_references,survey_typhoon_status,survey_typhoon_instrument,survey_typhoon_notes,survey_typhoon_references,survey_hst_status,survey_hst_instrument,survey_hst_notes,survey_hst_references,survey_irac_status,survey_irac_instrument,survey_irac_notes,survey_irac_references,survey_mips_status,survey_mips_instrument,survey_mips_notes,survey_mips_references,survey_wise_status,survey_wise_instrument,survey_wise_notes,survey_wise_references,survey_herschel_status,survey_herschel_instrument,survey_herschel_notes,survey_herschel_references,survey_alma_status,survey_alma_instrument,survey_alma_notes,survey_alma_references,survey_otherco_status,survey_otherco_instrument,survey_otherco_notes,survey_otherco_references,survey_hiresco_status,survey_hiresco_instrument,survey_hiresco_notes,survey_hiresco_references,survey_multijco_status,survey_multijco_instrument,survey_multijco_notes,survey_multijco_references,survey_dense_status,survey_dense_instrument,survey_dense_notes,survey_dense_references,survey_hi_status,survey_hi_instrument,survey_hi_notes,survey_hi_references,orient_ra,orient_ra_unc,orient_dec,orient_dec_unc,orient_pos_ref,orient_vlsr,orient_vlsr_unc,orient_vlsr_ref,orient_incl,orient_incl_unc,orient_posang,orient_posang_unc,orient_ref,rotcur_v0,rotcur_v0_unc,rotcur_rt,rotcur_rt_unc,rotcur_ref,dist,dist_unc,dist_label,dist_ref,size_r90,size_r90_unc,size_r90_ref,mwext_sfd98,mwext_sfd98_unc,mwext_sf11,mwext_sf11_unc,z0mgs_mtol,props_mtol_method,props_sfr,props_sfr_unc,props_sfr_method,props_mstar,props_mstar_unc,props_mtol,props_deltams,props_deltams_unc,props_mhi,props_mhi_unc,lco_phangs,lco_phangs_unc,appcor_phangs,aco10_phangs,size_r25,size_r25_unc,size_reff,size_reff_unc,size_scalelength,size_scalelength_unc,mh2_phangs,mh2_phangs_unc,morph_t,morph_t_unc,morph_string,morph_bar,morph_buta15,morph_class,morph_gold,morph_bar_flag,morph_bar_axis_ratio,morph_bar_pa,morph_bar_r,morph_bar_red,morph_bar_q_flag,morph_nuclear_bar_axis_ratio,morph_nuclear_bar_pa,morph_nuclear_bar_r,morph_spiral_arms,morph_multi_arm,morph_featureless,morph_disk_1_axis_ratio,morph_disk_1_pa,morph_disk_1_r,morph_disk_2_axis_ratio,morph_disk_2_pa,morph_disk_2_r,morph_bulge_axis_ratio,morph_bulge_pa,morph_bulge_r,morph_rings_outer,morph_ring_inner,morph_lens_1_axis_ratio,morph_lens_1_pa,morph_lens_1_r,morph_lens_2_axis_ratio,morph_lens_2_pa,morph_lens_2_r,morph_ring_1_axis_ratio,morph_ring_1_pa,morph_ring_1_r,morph_ring_2_axis_ratio,morph_ring_2_pa,morph_ring_2_r,morph_vcc,morph_interacting,morph_in_virgo_fornax,morph_high_peculiar,morph_comments,agn_veron_y_n,agn_veron_class,agn_milliquas_y_n,agn_milliquas_class_pQSO,props_mmol
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,arcsec,deg,arcsec,,km / s,km / s,,deg,deg,deg,deg,,km / s,km / s,arcsec,arcsec,,Mpc,,,,arcsec,arcsec,,mag,mag,mag,mag,solMass / solLum,,solMass / yr,solMass / yr,,solMass,solMass,,,,solMass,solMass,pc2 K km / s,pc2 K km / s,,s solMass / (K km pc2),arcsec,arcsec,arcsec,arcsec,arcsec,arcsec,solMass,solMass,,,,,,,,,,deg,arcsec,,,,deg,arcsec,,,,,deg,arcsec,,deg,arcsec,,deg,arcsec,,,,deg,arcsec,,deg,arcsec,,deg,arcsec,,deg,arcsec,,,,,,,,,,solMass
bytes100,int64,bytes1000,bytes100,bytes100,bytes1000,bytes100,bytes100,bytes100,bytes1000,bytes100,bytes100,bytes100,bytes1000,bytes100,bytes100,bytes100,bytes1000,bytes100,bytes100,bytes100,bytes1000,bytes100,bytes100,bytes100,bytes1000,bytes100,bytes100,bytes100,bytes1000,bytes100,bytes100,byte

In [6]:
sample_table_cut = sample_table['name', 'orient_ra', 'orient_dec', 'orient_incl', 'orient_posang', 
                                'morph_string', 'dist', 'size_reff', 'orient_vlsr', 'props_mhi', 'props_mmol',  
                                'props_mstar', 'props_sfr']

for i, n in enumerate(sample_table_cut['name']):
    sample_table_cut['name'][i] = n.upper()
    
sample_table_cut['size_reff'] = sample_table_cut['size_reff']*sample_table_cut['dist']*1e6/206268*1e-3*au.kpc/au.arcsec

sample_table_cut['props_mhi'] = np.log10(sample_table_cut['props_mhi'])
sample_table_cut['props_mstar'] = np.log10(sample_table_cut['props_mstar'])
sample_table_cut['props_mmol'] = np.log10(sample_table_cut['props_mmol'])
# sample_table_cut['props_sfr'] = np.log10(sample_table_cut['props_sfr'])

In [7]:
sample_table_cut

name,orient_ra,orient_dec,orient_incl,orient_posang,morph_string,dist,size_reff,orient_vlsr,props_mhi,props_mmol,props_mstar,props_sfr
,deg,deg,deg,deg,,Mpc,kpc,km / s,solMass,solMass,solMass,solMass / yr
bytes100,float32,float32,float32,float32,bytes100,float32,float64,float32,float32,float32,float32,float32
NGC3627,170.06252,12.9915,57.3,173.1,Sb,11.32,3.6363277289739564,715.35803,9.092605,9.777565,10.832936,3.8447642
NGC4321,185.72887,15.822304,38.5,156.2,SABb,15.21,5.499487346558846,1572.3125,9.43237,9.89057,10.74512,3.5552816
NGC6946,308.71918,60.153515,33.0,243.0,SABc,7.34,4.467678941959005,61.289387,10.022305,nan,10.467179,5.857527


In [8]:
names = ['Galaxy',
         'RA',
         'DEC',
         '$i$',
         'PA',
         'Morph.',
         'Dist.',
         '$R_\mathrm{eff}$',
         '$V_\mathrm{LSR}$',
         '$M_\mathrm{H_I}$',
         '$M_\mathrm{CO}$',
         '$M_\mathrm{star}$',
         'SFR']

formats={
        'RA': '%0.3f',
        'DEC': '%0.3f',
        'Dist.': '%0.1f', 
        '$R_\mathrm{eff}$': '%0.1f',
        '$V_\mathrm{LSR}$': '%0.1f',
        '$M_\mathrm{H_I}$': '%0.1f',
        '$M_\mathrm{CO}$': '%0.1f',
        '$M_\mathrm{star}$': '%0.1f',
        'SFR': '%0.1f'}

sample_table_cut.write('./tabs/sourceprops_tmp.tex', formats=formats, names=names, overwrite=True)